In [ ]:
import csv
import json
import shutil
import os
import random

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Data/txt/")

!pip install twarc
from twarc import Twarc 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
at_lst = [["E1YCRUZzmnTszjGTvVNgBzuH4", "4bEluZ46osuYZKsR3utXekRFbeAKQ9ZdfpaYfZFD6N6MwqioXC", \
                "1337206592941469697-gMJskxGtsoCo0oPg8RnDtMd8vZXSK1", "XWKvo7kCER5Grdvn4zNL3XP0AUERItI5Liigtb8oDNLUC"], \
                ["Xz47Kc1Wim3J9NNJqzXhpl2dD", "ASooOEwXQRI2M26mlU3PJZoFansCdPg4wWxv79D21xxPPDhiqN", \
                "869437095898689536-a1mS6NgGRj0IJiE7Nod1qLsZd3IE5HN", "Cq7aTQ0H9ZFEhakM40h8AdzOCbTMJXEiPnIicLj5ieyfP"]]

at_key = random.choice(at_lst)
consumer_key, consumer_secret = at_key[0], at_key[1]
access_token, access_token_secret = at_key[2], at_key[3]
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

In [ ]:
def check_key(json, key):
    try:
        rt = json[key]
    except KeyError:
        return False
    return True

In [ ]:
Mondt = {"02":29, "03":31, "04":30, "05":1}
path_csv = "/content/drive/MyDrive/Data/csv/"

for key, value in Mondt.items():
    for day_i in range(1, value+1):
        if day_i < 10:
            day_j = '0' + f'{day_i}'
        else:
            day_j = f'{day_i}'
        
        f_name = 'geo_2020-'+ key + '-' + day_j
        f_name_txt = 'text_' + f_name + '.txt'
        f_name_json = 'temp_' + f_name + '.json'
        f_name_csv = 'hy_' + f_name + '.csv'

        txtfile = open('./' + f_name + '.txt')
        lines = txtfile.readlines()

        ids = []
        for line in lines:
            ids.append(line.split(',')[0].strip())
        
        with open(f_name_txt, "w", encoding='utf-8-sig') as fout:
            print(*ids, sep="\n", file = fout)
        
        with open(f_name_csv, "a+", newline="", encoding='utf-8-sig') as csvfile:
            writer = csv.writer(csvfile)
            with open(f_name_txt, encoding='utf-8-sig') as txtid:
                for tweet in t.hydrate(txtid):
                    row = []
                    row.append(tweet['created_at'])
                    row.append(tweet['id'])
                    if check_key(tweet, 'retweeted_status') == False:
                        row.append(tweet['full_text'])
                        row.append(tweet['entities']['hashtags'])
                    else:
                        row.append(tweet['retweeted_status']['full_text'])
                        row.append(tweet['retweeted_status']['entities']['hashtags'])
                    row.append(tweet['user']['location'])
                    row.append(tweet['geo'])
                    row.append(tweet['coordinates'])
                    row.append(tweet['place'])
                    row.append(tweet['retweet_count'])
                    row.append(tweet['favorite_count'])
                    row.append(tweet['lang'])
                    writer.writerow(row)
        shutil.move(f_name_csv, path_csv + f_name_csv)
        os.remove(f_name_txt)